In [1]:
%%script python --bg
from rtCommon.projectInterface import Web
from rtCommon.structDict import StructDict

scriptToRun = 'projects/sample/sample.py'
initScript = 'projects/sample/initialize.py'
finalizeScript = 'projects/sample/finalize.py'
#defaultConfig = 'projects/sample/conf/sample.toml'

config = StructDict({
    'title' : 'Sample Project',
    'sessionId' : '20180310T123000',
    'subjectName' : '0219191_faceMatching',
    'datestr' : '20190219',
    'subjectNum' : 101,
    'subjectDay' : 1,
    'sessionNum' : 1,
    'runNum' : [1],
    'scanNum' : [13],

    ## Plotting settings
    'plotTitle' : 'Realtime Plot',
    'plotXLabel' : 'Sample #',
    'plotYLabel' : 'Value',
    'plotXRangeLow' : 0,
    'plotXRangeHigh' : 20,
    'plotYRangeLow' : -1,
    'plotYRangeHigh' : 1,
    'plotAutoRangeX' : False,
    'plotAutoRangeY' : True,

    ## important variables about the dicom files
    'dicomNamePattern' : '001_0000{}_000{}.dcm',
    'minExpectedDicomSize' : 300000,
})

params = StructDict({
    'fmriPyScript': scriptToRun,
    'initScript': initScript,
    'finalizeScript': finalizeScript,
    'filesremote': False,
    'port': 8889
})

web = Web()
web.start(params, config, testMode=True)

In [2]:
%%html
<iframe src="http://localhost:8889" width="800" height="600"></iframe>